<a href="https://colab.research.google.com/github/StonecoldLi/Kaggle_Contest_Samli/blob/main/Natural%20Language%20Processing%20with%20Disaster%20Tweets/Bert_finetune_official_version_basedonhuggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Tue May 23 14:48:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install -q kaggle

读取竞赛数据集

In [5]:
#上传凭证
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jfli318","key":"475ac384f8639d8bb024bc7891fec449"}'}

In [6]:
#create a kaggle folder
!mkdir ~/.kaggle

In [7]:
#copy the kaggle.json to folder created
!cp kaggle.json ~/.kaggle/

In [8]:
# permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle config set -n path -v .

- path is now set to: .


In [10]:
!kaggle competitions download -c nlp-getting-started

100% 593k/593k [00:00<00:00, 748kB/s]
100% 593k/593k [00:00<00:00, 748kB/s]


In [11]:
!unzip '/content/competitions/nlp-getting-started/nlp-getting-started.zip' -d '.'

Archive:  /content/competitions/nlp-getting-started/nlp-getting-started.zip
  inflating: ./sample_submission.csv  
  inflating: ./test.csv              
  inflating: ./train.csv             


## Install the basic libraries

In [12]:
!pip install transformers==4.28.0 datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [14]:
from huggingface_hub import notebook_login

notebook_login()

## Set Dataset
  - set the dataset into 2 columns 1 is text, 1 is label
  - we suppose that Location and Keyword make no sense in this task at first

In [15]:
import pandas as pd

In [16]:
# take a look at one example training data (text)
train = pd.read_csv("train.csv")
train['text'][1],train['target'][1]

('Forest fire near La Ronge Sask. Canada', 1)

In [17]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [18]:
def make_csv(df):
  df1 = df.drop(columns=['location','keyword','id'])
  return df1

In [19]:
train1 = make_csv(train)
train1.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [20]:
train1.rename(columns={'target':'label'},inplace=True)
train1.head()

,text,label
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [21]:
train1.to_csv('train_text.csv')

In [22]:
# take a look at one example tresting data (text)
test = pd.read_csv("test.csv")
test['text'][1]

'Heard about #earthquake is different cities, stay safe everyone.'

In [23]:
test1 = make_csv(test)
test1.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [24]:
test1.to_csv('test_text.csv')

## Preprocess 预处理

In [25]:
from transformers import AutoTokenizer

In [26]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") #指定tokenizer类型

In [27]:
# def preprocess_function(examples):
#   return tokenizer(examples["text"], truncation=True)

def preprocess_function(example):
    text = example["text"]
    label = example["label"]
    tokenized_text = tokenizer(text, truncation=True)
    return {
        "text": text,
        "label": label,
        "input_ids": tokenized_text["input_ids"],
        "attention_mask": tokenized_text["attention_mask"]
    }

In [28]:
from sklearn.model_selection import train_test_split
train_train_data, train_test_data = train_test_split(train1, test_size=0.1, random_state=318)

In [29]:
train_examples = train_train_data.to_dict("records")
tokenized_train = [preprocess_function(example) for example in train_examples]

In [30]:
tokenized_train[1]

{'text': 'Weather forecast for Thailand  A Whirlwind is coming ...2 september https://t.co/rUKjYjG9oQ',
 'label': 1,
 'input_ids': [101,
  4633,
  19939,
  2005,
  6504,
  1037,
  1059,
  11961,
  2140,
  11101,
  2003,
  2746,
  1012,
  1012,
  1012,
  1016,
  2244,
  16770,
  1024,
  1013,
  1013,
  1056,
  1012,
  2522,
  1013,
  21766,
  2243,
  3501,
  2100,
  3501,
  2290,
  2683,
  2080,
  4160,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [31]:
test_examples = train_test_data.to_dict("records")
tokenized_test = [preprocess_function(example) for example in test_examples]

In [32]:
tokenized_test[1]

{'text': '@Jude_Mugabi not that all abortions get you traumatised. At times you are okay with the decision due to reasons like rape',
 'label': 0,
 'input_ids': [101,
  1030,
  12582,
  1035,
  14757,
  28518,
  2025,
  2008,
  2035,
  11324,
  2015,
  2131,
  2017,
  12603,
  7315,
  2098,
  1012,
  2012,
  2335,
  2017,
  2024,
  3100,
  2007,
  1996,
  3247,
  2349,
  2000,
  4436,
  2066,
  9040,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [33]:
#create a batch of examples using DataCollatorWithPadding
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [34]:
import evaluate

accuracy = evaluate.load("accuracy") #加载评价指标方法

In [35]:
#create a function that passes your predictions and labels to compute to calculate the accuracy
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

In [36]:
id2label = {0: int("0"), 1: int("1")}
label2id = {int("0"):0,int("1"):1}

In [37]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [38]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/StonecoldLi/my_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 7.40k/255M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Download file runs/May23_14-16-46_c8f98576cb3f/events.out.tfevents.1684851412.c8f98576cb3f.1311.0: 100%|######…

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

...

Clean file runs/May23_14-16-46_c8f98576cb3f/1684851412.3412452/events.out.tfevents.1684851412.c8f98576cb3f.131…

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.404196,0.830709
2,No log,0.382244,0.843832
3,No log,0.394863,0.838583
4,No log,0.384598,0.841207
5,0.368600,0.397816,0.841207
6,0.368600,0.410465,0.837270
7,0.368600,0.416631,0.838583
8,0.368600,0.424319,0.837270
9,0.368600,0.430550,0.839895
10,0.234200,0.433330,0.838583


TrainOutput(global_step=1080, training_loss=0.29411527668988263, metrics={'train_runtime': 593.2848, 'train_samples_per_second': 115.476, 'train_steps_per_second': 1.82, 'total_flos': 1092114273224604.0, 'train_loss': 0.29411527668988263, 'epoch': 10.0})

## Inference

In [39]:
test1['text'][1]

'Heard about #earthquake is different cities, stay safe everyone.'

In [40]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./my_model")
inputs = tokenizer(test['text'][1], return_tensors="pt")

In [41]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("./my_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [42]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

1

In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("./my_model")
model = AutoModelForSequenceClassification.from_pretrained("./my_model")

# 假设 test 是一个包含句子的 Series 对象
test_sentences = test1['text']

# 将句子列表转换为模型输入的批量格式
inputs = tokenizer(test_sentences.tolist(), padding=True, truncation=True, return_tensors="pt")
batch_size = 128

# 将输入放入设备（GPU 或 CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}

# 将模型移动到相同的设备上
model = model.to(device)

# 批量预测
with torch.no_grad():
    logits = model(**inputs).logits

# 获取预测结果
predicted_class_ids = logits.argmax(dim=1).tolist()

# 获取预测标签
predicted_labels = [model.config.id2label[pred] for pred in predicted_class_ids]

# 输出结果
for label in predicted_labels:
    print(label)


In [47]:
test = pd.read_csv('test.csv')

In [49]:
pd.DataFrame({'id':test.id, 'target':predicted_labels}).set_index('id').to_csv('submission_distilbert-base-uncased_textonly_ver1.csv')

- 0.82623